In [2]:
!pip install selenium

In [34]:
#Import all libraries
import selenium
from selenium import webdriver
import pandas as pd
import warnings
import requests

# importing regex
import re
from selenium import webdriver

# Importing required Exceptions
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait

In [35]:
driver = webdriver.Chrome(r'C:\Users\Ricky Riddhi\Downloads\chromedriver_win32\chromedriver.exe')

#### Question 1&2. Write a python program which searches all the product under a particular product from www.amazon.in.

The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search
for guitars

Question 2 - In the above question, now scrape the following details of each product listed in first 3 pages of your 
search results and save it in a data frame and csv. In case if any product has less than 3 pages in search 
results then scrape all the products available under that product name. Details to be scraped are: "Brand 
Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and 
“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“

In [36]:
#Get webpage Amazon.in
url = 'https://www.amazon.in/'
driver.get(url)

In [37]:
search_product = driver.find_element_by_id('twotabsearchtextbox')

In [38]:
#Finding web element for search product bar using guitar
search_product.send_keys("Guitar")

In [39]:
submit = driver.find_element_by_id('nav-search-submit-button')
submit.click()

In [40]:
title =  driver.find_elements_by_xpath('//a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]')
len(title)
title[0:5]

[<selenium.webdriver.remote.webelement.WebElement (session="a94ea65b0cd12f2a65e374f75c204977", element="20c1858a-907c-4f21-9978-233a198267aa")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a94ea65b0cd12f2a65e374f75c204977", element="9f82a54f-3e10-44e3-bc4b-4aef88fb306e")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a94ea65b0cd12f2a65e374f75c204977", element="bed0b1a8-25dc-47a9-aebe-85b9d418b838")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a94ea65b0cd12f2a65e374f75c204977", element="1202bf55-01ae-4edb-9465-7c215d5ac1f1")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a94ea65b0cd12f2a65e374f75c204977", element="4aab4edc-03f6-46ac-ab5e-47624686bf72")>]

In [41]:
product = []

for i in title:
    product.append(i.text)
len(product)
product[0:10]

['Intern INT-38C Acoustic Guitar Kit, With Bag, Strings, Pick And Strap, Blue',
 'Photron Acoustic Guitar, 38 Inch Cutaway, PH38C/BK with Picks Only, Black (Without Bag, Strap and Extra Strings)',
 'Kadence Frontier guitar with Online Guitar learning course, Wine Red Acoustic Guitar with Die Cast Keys, Set of Strings, Strap, Picks and Bag (Wine Red, Acoustic)',
 'Kadence Frontier Jumbo Semi Acoustic Guitar With Die Cast Keys Super Combo (Bag, 1 pack Strings, Strap, Picks, Capo, Tuner (natural)',
 'Juarez Acoustic Guitar, 38 Inch Curved Body Cutaway, 38CUR with Bag, Strings, Pick and Strap, Black',
 'Juârez Acoustic Guitar, 38 Inch Cutaway, 038C with Bag, Strings, Pick and Strap, Black',
 'Intern INT-38C Acoustic Guitar Kit, With Bag, Strings, Pick And Strap, Blue',
 'Epiphone PRO-1 6 Strings Right handed Acoustic Guitar Vintage Sunburst',
 'Kadence Slowhand Premium Jumbo Semi Acoustic Guitar with Heavy Padded Bag, guitar cable, Pro Capo (Black Spruce Wood)',
 '']

In [61]:
import time
urls = []
for page in range(0,3):

    try:
        page_urls = driver.find_elements_by_xpath('//a[@class="a-link-normal s-no-outline"]')
        
        for url in page_urls:
            url = url.get_attribute('href')     
            if url[0:4]=='http':                
                urls.append(url)                
        print("Product urls of page {} has been scraped.".format(page+1))
        
        nxt_button = driver.find_elements_by_xpath('//li[@class="a-last"]/a')      
        if nxt_button.text == 'Next→':                                            
            nxt_button.click()                                                    
            time.sleep(5)                                                            
        elif driver.find_elements_by_xpath('//li[@class="a-disabled a-last"]/a').text == 'Next→':    
            print("No new pages exist") 
            break
            
    except NoSuchElementException as e:             
        print("NoSuchElement Exception")
        next_page = nxt_button.get_attribute('href')        
        driver.get(next_page)

Product urls of page 1 has been scraped.


AttributeError: 'list' object has no attribute 'text'

In [59]:
len(urls)

64

In [46]:
#Making empty lists and scraping the required spots
brand_name=[]
product_name=[]
product_rating=[]
product_price=[]
product_return=[]
product_delivery=[]
product_availability=[]
product_url=[]

for url in urls:
    driver.get(url)                                                     
    print("Scraping URL = ", url)
    
    try:
        brand = driver.find_element_by_xpath('//a[@id="bylineInfo"]')      
        brand_name.append(brand.text)
    except NoSuchElementException:
        brand_name.append('-')
    
    try:
        name = driver.find_element_by_xpath('//h1[@id="title"]/span')      
        product_name.append(name.text)
    except NoSuchElementException:
        product_name.append('-')
    
    try:
        rating = driver.find_element_by_xpath('//span[@id="acrPopover"]')  
        product_rating.append(rating.get_attribute("title"))
    except NoSuchElementException:
        product_rating.append('-')
    
    try:
        price = driver.find_element_by_xpath('//span[@id="priceblock_ourprice"]')  
        product_price.append(price.text)
    except NoSuchElementException:
        product_price.append('-')
    
    try:                                                                                     
        ret = driver.find_element_by_xpath('//div[@data-name="RETURNS_POLICY"]/span/div[2]/a')
        product_return.append(ret.text)
    except NoSuchElementException:
        product_return.append('-')
    
    try:
        delivry = driver.find_element_by_xpath('//div[@id="ddmDeliveryMessage"]/b')         
        product_delivery.append(delivry.text)
    except NoSuchElementException:
        product_delivery.append('-')
    
    try:
        avl = driver.find_element_by_xpath('//div[@id="availability"]/span')        
        product_availability.append(avl.text)
    except NoSuchElementException:
        product_availability.append('-')
    
    product_url.append(url)                                                            
    time.sleep(2)

Scraping URL =  https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A0266550G4QCJGMMKGHT&url=%2FIntern-INT-38C-Acoustic-Guitar-Blue%2Fdp%2FB073B5RZBV%2Fref%3Dsr_1_1_sspa%3Fcrid%3D2I3HWBNYOQNRN%26keywords%3DGuitar%26qid%3D1649015524%26sprefix%3Dguitar%252Caps%252C418%26sr%3D8-1-spons%26psc%3D1&qualifier=1649015524&id=5611581234389295&widgetName=sp_atf
Scraping URL =  https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A06072772I2LJ344EB41Y&url=%2FPhotron-Acoustic-Cutaway-PH38C-BK%2Fdp%2FB076T8V8NB%2Fref%3Dsr_1_2_sspa%3Fcrid%3D2I3HWBNYOQNRN%26keywords%3DGuitar%26qid%3D1649015524%26sprefix%3Dguitar%252Caps%252C418%26sr%3D8-2-spons%26psc%3D1&qualifier=1649015524&id=5611581234389295&widgetName=sp_atf
Scraping URL =  https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A061107531TUGN5RHMV0N&url=%2FKadence-Frontier-Acoustic-Guitar-Strings%2Fdp%2FB01GDZ46AA%2Fre

In [49]:
data=list(zip(brand_name,product_name,product_rating,product_price,product_return,product_delivery,product_availability,product_url))

In [51]:
df=pd.DataFrame(data,columns=['Brand','Product_Name','Rating','Price','Product_Return','Product_Delivery','Availability','Product_URL'])
df

,Brand,Product_Name,Rating,Price,Product_Return,Product_Delivery,Availability,Product_URL
0,Visit the Intern Store,"Intern INT-38C Acoustic Guitar Kit, With Bag, ...",4.0 out of 5 stars,-,7 Days Replacement,-,Only 1 left in stock (more on the way).,https://www.amazon.in/gp/slredirect/picassoRed...
1,Brand: Photron,"Photron Acoustic Guitar, 38 Inch Cutaway, PH38...",3.9 out of 5 stars,-,7 Days Replacement,-,In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
2,Visit the Kadence Store,Kadence Frontier guitar with Online Guitar lea...,4.0 out of 5 stars,-,7 Days Replacement,"Wednesday, April 6",,https://www.amazon.in/gp/slredirect/picassoRed...
3,Visit the Kadence Store,Kadence Frontier Jumbo Semi Acoustic Guitar Wi...,4.0 out of 5 stars,-,7 Days Replacement,-,In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
4,Visit the JUAREZ Store,"Juarez Acoustic Guitar, 38 Inch Curved Body Cu...",4.0 out of 5 stars,-,7 Days Replacement,-,In stock.,https://www.amazon.in/Juarez-Acoustic-Guitar-C...
...,...,...,...,...,...,...,...,...
59,Brand: Generic,Signature original Acoustic Guitar Orange colo...,-,-,7 Days Replacement,-,,https://www.amazon.in/Signature-original-Acous...
60,Visit the VAULT Store,Vault Traveller 34 Inch Acoustic Guitar With B...,3.9 out of 5 stars,-,7 Days Replacement,-,In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
61,Visit the blueberry Store,"Blueberry 38C IN Acoustic Guitar Kit with Bag,...",3.8 out of 5 stars,-,7 Days Replacement,-,In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
62,Brand: HOMME STORE,HOMME STORE 4 String Guitar Children's Musical...,2.5 out of 5 stars,-,7 Days Replacement,-,In stock.,https://www.amazon.in/gp/slredirect/picassoRed...


#### Question 3.Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [68]:
#Import all libraries
import selenium
from selenium import webdriver
import pandas as pd
import warnings
import requests

# importing regex
import re
from selenium import webdriver

# Importing required Exceptions
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait

# Connecting to the webdriver
driver=webdriver.Chrome(r'C:\Users\Ricky Riddhi\Downloads\chromedriver_win32\chromedriver.exe')

In [69]:
url = "https://images.google.com/"
#Creating empty list 
urls = []    #empty list
data = []    #empty list
search_item = ["fruits", "cars", "Machine Learning", "Guitar", "Cakes"]
for item in search_item:
    driver.get(url)  
    time.sleep(5)
    
    # Finding webelement for search_bar
    search_bar = driver.find_element_by_tag_name("input") 
    
    # Sending keys to get the keyword for search bar
    search_bar.send_keys(str(item))
    
    # Clicking on search button
    search_button =driver.find_element_by_xpath("//button[@class='Tg7LZd']").click()
    
    # Scrolling down the webpage to get some more images
    for _ in range(500):
        driver.execute_script("window.scrollBy(0,100)")
        
        imgs = driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']")
    img_url = []
    for image in imgs:
        source = image.get_attribute('src')
        if source is not None:
                if(source[0:4] == 'http'):
                    img_url.append(source)
    for i in img_url[:100]:
        urls.append(i)
                    
for i in range(len(urls)):
    if i >= 500:
        break
    print("Downloading {0} of {1} images" .format(i, 500))
    response = requests.get(urls[i])

#### Question 4: Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on
www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be 
scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, 
“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the 
details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [70]:
driver=webdriver.Chrome(r'C:\Users\Ricky Riddhi\Downloads\chromedriver_win32\chromedriver.exe')

In [71]:
url = 'https://www.flipkart.com/'
driver.get(url)

In [72]:
# Closing Login popup button
login_X_btn = driver.find_element_by_xpath("//div[@class='_2QfC02']//button").click()

In [73]:
# Searching for web element
search_bar = driver.find_element_by_xpath("//input[@class='_3704LK']")
# Sending keys to search product
search_bar.send_keys("pixel 4A")

In [74]:
# Locating the search button using xpath
search_btn = driver.find_element_by_xpath("//button[@class='L0Z3Pu']")
# Clicking on search button
search_btn.click()

In [75]:
# Fetching 1st page URLs of smartphone
page1_url = []   #empty list
urls = driver.find_elements_by_xpath("//a[@class='_1fQZEK']")
for url in urls:
    page1_url.append(url.get_attribute('href'))

In [76]:
len(page1_url)

24

In [77]:
Smartphones = {}
Smartphones["Brand"] = []
Smartphones["Phone name"] = []
Smartphones["Colour"] = []
Smartphones["RAM"] = []
Smartphones["Storage(ROM)"] = []
Smartphones["Primary Camera"] = []
Smartphones["Secondary Camera"] = []
Smartphones["Display Size"] = []
Smartphones["Battery Capacity"] = []
Smartphones["Price"] = []
Smartphones["URL"] = []

In [78]:
# Scraping data from each url of page 1
for url in page1_url:
    driver.get(url)                                                        
    print("Scraping URL = ", url)
    Smartphones['URL'].append(url)                                                          
    time.sleep(2)
    
    
    #Clicking on read more button to get more information
    try:
        read_more = driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]')     
        read_more.click()
    except NoSuchElementException:
        print("Exception occured while moving to next page")
    
    
    #Scraping brand name of smartphone
    try:
        brand_tags = driver.find_element_by_xpath('//span[@class="B_NuCI"]')      
        Smartphones["Brand"].append(brand_tags.text.split()[0])
    except NoSuchElementException:
        Smartphones['Brand'].append('-')
    
    
    #Scraping name of smartphone
    try:
        name_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][1]/table/tbody/tr[3]/td[2]/ul/li')     
        Smartphones['Phone name'].append(name_tags.text)
    except NoSuchElementException:
        Smartphones['Phone name'].append('-')
    
    
    #Scraping colour of smartphone
    try:
        color_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][1]/table/tbody/tr[4]/td[2]/ul/li')      
        Smartphones['Colour'].append(color_tags.text)
    except NoSuchElementException:
        Smartphones['Colour'].append('-')
     
    
    #Scraping RAM data of smartphone
    try:
        ram_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[2]/td[2]/ul/li')                
        Smartphones['RAM'].append(ram_tags.text)
    except NoSuchElementException:
        Smartphones['RAM'].append('-')
    
    
    #Scraping ROM data of smartphone
    try:
        rom_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[1]/td[2]/ul/li')        
        Smartphones['Storage(ROM)'].append(rom_tags.text)
    except NoSuchElementException:
        Smartphones['Storage(ROM)'].append('-')
        
        
    #Scraping Primary camera data of smartphone
    try:                                                                                    
        pri_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[2]/td[2]/ul/li')
        Smartphones['Primary Camera'].append(pri_tags.text)
    except NoSuchElementException:
        Smartphones['Primary Camera'].append('-')
        
        
    #Scraping secondary camera data of smarphone
    try:                                                                                    
        sec_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[1]')
        if sec_tags != "Secondary Camera" : 
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[1]').text == "Secondary Camera":
                sec_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[2]/ul/li')
            else :
                raise NoSuchElementException
        else :
            sec_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[2]/ul/li')
        Smartphones['Secondary Camera'].append(sec_cam.text)
    except NoSuchElementException:
        Smartphones['Secondary Camera'].append('-')
        
        
    #Scraping Display size data of smartpone
    try:
        disp_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/div')
        if disp_tags.text != "Display Features" : raise NoSuchElementException
        disp_size = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[1]/td[2]/ul/li')  
        Smartphones['Display Size'].append(disp_size.text)
    except NoSuchElementException:
        Smartphones['Display Size'].append('-')   
    
       
    
     #Scraping battery capacity of smartphone
    try:
        if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                bat_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[1]')
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[2]/ul/li')                
            elif driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :
                bat_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[1]')
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[1]')
            if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
            bat_capa = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[2]/ul/li')              
        Smartphones['Battery Capacity'].append(bat_capa.text)
    except NoSuchElementException:
        Smartphones['Battery Capacity'].append('-')
        
        
        
    #Scraping Price of smartphone
    try:
        price_tags = driver.find_element_by_xpath('//div[@class="_30jeq3 _16Jk6d"]')      
        Smartphones['Price'].append(price_tags.text)
    except NoSuchElementException:
        Smartphones['Price'].append('-')

Scraping URL =  https://www.flipkart.com/google-pixel-4a-just-black-128-gb/p/itm023b9677aa45d?pid=MOBFUSBNAZGY7HQU&lid=LSTMOBFUSBNAZGY7HQUWHTF0C&marketplace=FLIPKART&q=pixel+4A&store=tyy%2F4io&srno=s_1_1&otracker=search&otracker1=search&fm=Search&iid=99a30678-e05d-498a-b66f-8e411f701452.MOBFUSBNAZGY7HQU.SEARCH&ppt=hp&ppn=homepage&ssid=wchws40mw00000001649018470168&qH=9b26a23b2cff510d
Scraping URL =  https://www.flipkart.com/redmi-9a-sport-metallic-blue-32-gb/p/itm53bc6ebfe147b?pid=MOBG7GY6GGZ8XZGM&lid=LSTMOBG7GY6GGZ8XZGMYMD4P9&marketplace=FLIPKART&q=pixel+4A&store=tyy%2F4io&srno=s_1_2&otracker=search&otracker1=search&fm=Search&iid=99a30678-e05d-498a-b66f-8e411f701452.MOBG7GY6GGZ8XZGM.SEARCH&ppt=hp&ppn=homepage&ssid=wchws40mw00000001649018470168&qH=9b26a23b2cff510d
Scraping URL =  https://www.flipkart.com/redmi-9a-sport-carbon-black-32-gb/p/itm53bc6ebfe147b?pid=MOBG7CJMHFQJFCX3&lid=LSTMOBG7CJMHFQJFCX3ITLYP2&marketplace=FLIPKART&q=pixel+4A&store=tyy%2F4io&srno=s_1_3&otracker=search&otrac

In [79]:
#Framing the data
df = pd.DataFrame.from_dict(Smartphones)
df

,Brand,Phone name,Colour,RAM,Storage(ROM),Primary Camera,Secondary Camera,Display Size,Battery Capacity,Price,URL
0,Google,Pixel 4a,Just Black,6 GB,128 GB,12.2MP Rear Camera,8MP Front Camera,14.76 cm (5.81 inch),3140 mAh,"₹31,999",https://www.flipkart.com/google-pixel-4a-just-...
1,Redmi,Redmi 9A Sport,Metallic Blue,2 GB,32 GB,13MP Rear Camera,5MP Front Camera,16.59 cm (6.53 inch),5000 mAh,"₹7,098",https://www.flipkart.com/redmi-9a-sport-metall...
2,Redmi,Redmi 9A Sport,Carbon Black,3 GB,32 GB,13MP Rear Camera,5MP Front Camera,16.59 cm (6.53 inch),5000 mAh,"₹8,179",https://www.flipkart.com/redmi-9a-sport-carbon...
3,Redmi,Redmi 9A Sport,Carbon Black,2 GB,32 GB,13MP Rear Camera,5MP Front Camera,16.59 cm (6.53 inch),5000 mAh,"₹6,945",https://www.flipkart.com/redmi-9a-sport-carbon...
4,Redmi,Redmi 9A Sport,Metallic Blue,3 GB,32 GB,13MP Rear Camera,5MP Front Camera,16.59 cm (6.53 inch),5000 mAh,"₹8,258",https://www.flipkart.com/redmi-9a-sport-metall...
5,I,k6300 NEW,Blue,32 MB,64 MB,Primary Camera,-,7.11 cm (2.8 inch),3000 mAh,"₹1,199",https://www.flipkart.com/kall-k6300-new/p/itm1...
6,Redmi,Redmi Note 11,Space Black,6 GB,64 GB,Primary Camera,-,16.33 cm (6.43 inch),-,"₹14,599",https://www.flipkart.com/redmi-note-11-space-b...
7,Redmi,Redmi Note 11,Starburst White,6 GB,64 GB,Primary Camera,-,16.33 cm (6.43 inch),-,"₹15,639",https://www.flipkart.com/redmi-note-11-starbur...
8,Redmi,Redmi Note 11,Space Black,4 GB,64 GB,50MP + 8MP + 2MP + 2MP,13MP Front Camera,16.33 cm (6.43 inch),5000 mAh,"₹14,200",https://www.flipkart.com/redmi-note-11-space-b...
9,Redmi,Redmi Note 11,Horizon Blue,4 GB,64 GB,50MP + 8MP + 2MP + 2MP,13MP Front Camera,16.33 cm (6.43 inch),5000 mAh,"₹14,128",https://www.flipkart.com/redmi-note-11-horizon...


In [81]:
# Saving data in csv
df.to_csv("Flipkart_Smartphone.csv")

#### Question 5.Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [82]:
driver=webdriver.Chrome(r'C:\Users\Ricky Riddhi\Downloads\chromedriver_win32\chromedriver.exe')

In [83]:
# Getting mentioned url and opening google maps web page
url = "https://www.google.co.in/maps"
driver.get(url)
time.sleep(2)

# entering the city name in search bar 
City = input('Enter City name that has to be searched : ')
search_bar = driver.find_element_by_id("searchboxinput")                       
search_bar.clear()                                                             
time.sleep(2)

# Sending keys to find cities
search_bar.send_keys(City) 

# Checking for webelement and Clicking on search button
search_btn = driver.find_element_by_id("searchbox-searchbutton")              
search_btn.click()                                                             
time.sleep(2)

try:
    url_str = driver.current_url
    print("URL Extracted: ", url_str)
    latitude_longitude = re.findall(r'@(.*)data',url_str)
    if len(latitude_longitude):
        lat_lng_list = latitude_longitude[0].split(",")
        if len(lat_lng_list)>=2:
            latitude = lat_lng_list[0]
            longitude = lat_lng_list[1]
        print("Latitude = {}, Longitude = {}".format(latitude, longitude))

except Exception as e:
        print("Error: ", str(e))

Enter City name that has to be searched : Delhi
URL Extracted:  https://www.google.co.in/maps/place/Delhi/@28.573696,77.3849088,12z/data=!4m5!3m4!1s0x390cfd5b347eb62d:0x37205b715389640!8m2!3d28.7040592!4d77.1024902
Latitude = 28.573696, Longitude = 77.3849088


In [ ]:
Question 6. Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) from trak.in.

In [84]:
driver=webdriver.Chrome(r'C:\Users\Ricky Riddhi\Downloads\chromedriver_win32\chromedriver.exe')

In [85]:
# opening the url track.in
url = "https://trak.in/"
driver.get(url)
time.sleep(2)

In [86]:
#Getting xpath for funding deals and clicking on the button
fund_button = driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(fund_button)

In [87]:
# Empty lists
fund_deals = {}
fund_deals['Date'] = []
fund_deals['Startup Name'] = []
fund_deals['Industry/Vertical'] = []
fund_deals['Sub-Vertical'] = []
fund_deals['Location'] = []
fund_deals['Investor'] = []
fund_deals['Investment Type'] = []
fund_deals['Amount(in USD)'] = []


for i in range(54,57):

    # Scraping data of Date
    date = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for d in date:
        fund_deals['Date'].append(d.text)

    # Scraping data of Startup Name
    startup_name= driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for name in startup_name:
        fund_deals['Startup Name'].append(name.text)
    
    # Scraping data of Industry OR Vertical
    industry = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for ind in industry:
        fund_deals['Industry/Vertical'].append(ind.text)
    
    # Scraping data of Sub-Vertical
    sub_vertical = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for sv in sub_vertical:
        fund_deals['Sub-Vertical'].append(sv.text)

    # Scraping data of Location
    location = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for loc in location:
        fund_deals['Location'].append(loc.text)
    
    # Scraping data of Investor
    investor = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for invest in investor:
        fund_deals['Investor'].append(invest.text)
        
    # Scraping data of Investment Type
    investment_type = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for invtype in investment_type:
        fund_deals['Investment Type'].append(invtype.text)
        
    # Scraping data of Amount
    amount = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for amt in amount:
        fund_deals['Amount(in USD)'].append(amt.text)

In [88]:
# Creating DataFrame for scraped data
fund_data = pd.DataFrame(fund_deals)
fund_data

,Date,Startup Name,Industry/Vertical,Sub-Vertical,Location,Investor,Investment Type,Amount(in USD)
0,15/01/2021,Digit Insurance,Financial Services,Insurance Services,Bengaluru,"A91 Partners, Faering Capital, TVS Capital Funds",Venture,"1,80,00,000"
1,28/01/2021,Bombay Shaving Company,Consumer Goods Company,"Shave care, beard care, and skincare products",New Delhi,Reckitt Benckiser,Venture,"6,172,258.50"
2,19/01/2021,DeHaat,AgriTech Startup,online marketplace for farm products and services,Patna,Prosus Ventures,Series C,"30,000,000"
3,19/01/2021,Darwinbox,SaaS,HR Tech,Mumbai,Salesforce Ventures,Seed,"15,000,000"
4,18/01/2021,mfine,Health Tech Startup,AI-powered telemedicine mobile app,Bengaluru,Heritas Capital Management,Venture Round,"16,000,000"
5,18/01/2021,Udayy,EdTech,Online learning platform for kids in class 1-5,Gurgaon,Sequoia Capital,Seed Funding,"15,000,000"
6,11/01/2021,True Elements,Food Startup,Whole Food plant based Nashta,Pune,SIDBI Venture Capital,Series,"100,000,000"
7,13/01/2021,Saveo,B2B E-commerce,Pharmacies,Bengaluru,"Matrix Partners India, RTP Global, others",Seed,"4,000,000"
8,11/02/2021,Doubtnut,Edu Tech,E-Learning Platform,Gurgaon,"SIG Global, Sequoia Capital, WaterBridge Ventu...",Series B,"2,500,000"
9,22/02/2021,Zomato,Hospitality,Online Food Delivery Platform,Gurgaon,"Tiger Global, Kora",Venture,"250,000,000"


In [89]:
#Saving data in csv file
fund_data.to_csv("Trak_in.csv")

#### Question 7.Write a program to scrap all the available details of best gaming laptops from digit.in

In [90]:
driver=webdriver.Chrome(r'C:\Users\Ricky Riddhi\Downloads\chromedriver_win32\chromedriver.exe')

In [91]:
# opening the url digit.in
url = "https://www.digit.in/"
driver.get(url)
time.sleep(2)

In [92]:
#searching for best laptop
best_gam_laptops = driver.find_element_by_xpath("//div[@class='listing_container']//ul//li[9]").click()
time.sleep(4)

In [93]:
#Creating empty lists
Laptop_Name = []
Operating_sys = []
Display = []
Processor = []
Memory = []
Weight = []
Dimensions = []
Graph_proc = []
Price = []

In [94]:
# Scraping the data of laptop names
laptop_name= driver.find_elements_by_xpath("//div[@class='right-container']/div/a/h3")
for name in laptop_name:
    Laptop_Name.append(name.text)
    
    
#Scraping the data of operating system
try:
    op_sys = driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[1]/div/div")
    for os in op_sys:
        Operating_sys.append(os.text)
except NoSuchElementException:
    pass


#Scraping data of display of the laptop
try:
    display= driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[2]/div/div")
    for disp in display:
        Display.append(disp.text)
except NoSuchElementException:
    pass


#Scraping data of Processor
try:
    processor = driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[5]/td[3]")
    for pro in processor:
        Processor.append(pro.text)
except NoSuchElementException:
    pass

#Scraping data of memory
try:
    memory = driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[3]")
    for memo in memory:
        Memory.append(memo.text)
except NoSuchElementException:
    pass


#Scraping data of weight
try:
    weight = driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[7]/td[3]")
    for wgt in weight:
        Weight.append(wgt.text)
except NoSuchElementException:
    pass


#Scraping data of dimensions
try:
    dimension = driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[8]/td[3]")
    for dim in dimension:
        Dimensions.append(dim.text)
except NoSuchElementException:
    pass


#Scraping data of Graph processor
try:
    graph = driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[9]/td[3]")
    for gra in graph:
        Graph_proc.append(gra.text)
except NoSuchElementException:
    pass


#Scraping data of price
try:
    price = driver.find_elements_by_xpath("//td[@class='smprice']")
    for pri in price:
        Price.append(pri.text.replace('₹','Rs '))
except NoSuchElementException:
    pass

In [95]:
print(len(Laptop_Name),len(Operating_sys),len(Display),len(Processor),len(Memory),len(Weight),len(Dimensions),len(Graph_proc),len(Price))

10 10 10 10 10 10 10 10 10


In [96]:
data=list(zip(Laptop_Name, Operating_sys, Display, Processor, Memory, Weight, Dimensions, Graph_proc, Price))

In [97]:
df=pd.DataFrame(data,columns=['Laptop_Name','Operating_System','Display','Processor','Memory','Weight','Dimension','Graph_Proc','Price'])

In [98]:
df

,Laptop_Name,Operating_System,Display,Processor,Memory,Weight,Dimension,Graph_Proc,Price
0,HP VICTUS 16,WINDOWS 10,"16.1"" (1920 X 1200)",5th Gen AMD Ryzen 7 5800H | 3.2GHz,512 GB SSD/16 GBGB DDR4,2.48,37 x 26.2 x 2.4,NVIDIA GeForce RTX 3060,"Rs 104,990"
1,HP OMEN 15,WINDOWS 10 HOME,"15.6"" (1920 X 1080)",AMD Ryzen 7 Octa Core-5800H | 4.4 GHz,1 TB SSD/16 GBGB DDR4,2.37,357.9 x 239.7 x 22.5,NVIDIA GeForce RTX 3060,"Rs 110,790"
2,ACER NITRO 5,WINDOWS 10,"15.6"" (1920 X 1080)",AMD Ryzen 9 Octa Core | 2.4 GHz,1 TB HDD/16 GBGB DDR4,2.4,363.4 x 255 x 23.9,NVIDIA GeForce RTX 3070,"Rs 129,990"
3,MSI STEALTH 15M,WINDOWS 10,"15.6"" (1920 X 1080)",Intel Core i7 11th Gen - 11375H | NA,1 TB SSD/16 GBGB DDR4,1.7,358.3 x 248 x 16.15,NVIDIA GeForce RTX 3060,"Rs 134,990"
4,ASUS ROG STRIX SCAR 15,WINDOWS 10,"15.6"" (2560 X 1440)",AMD Ryzen 9 Octa Core - 5900HX | 3.3 GHz,2 TB SSD/32 GBGB DDR4,2.30,354 x 259 x 22.6,NVIDIA GeForce RTX 3080,"Rs 193,990"
5,ASUS ROG STRIX SCAR 15,WINDOWS 10 HOME,"15.6"" (1920 X 1080)",AMD Ryzen™ 9 5900HX | 3.3 GHz,1 TB SSD/16 GBGB DDR4,2.30,35.4 x 25.9 x 2.26,NVIDIA® GeForce RTX™ 3070,"Rs 215,990"
6,ASUS ZEPHYRUS G14,WINDOWS 10 HOME,"14"" (1920 X 1080)",AMD 3rd Gen Ryzen 9 | 3.3 GHz,1 TB SSD/16 GBGB DDR4,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060,"Rs 144,990"
7,LENOVO LEGION 5I,WINDOWS 10 PRO,"15.6"" (1920 X 1080)",10th Gen Intel® Core™ i5-10300H | 2.50 GHz,1 TB SSD/16 GBGB DDR4,2.3,363.06 x 259.61 x 23.57,NVIDIA® GeForce® GTX 1650 4GB,"Rs 76,988"
8,ASUS ROG ZEPHYRUS DUO 15,WINDOWS 10,"15.6"" (3840 X 1100)",Intel Core i7 10th Gen 10875H | NA,512 GB SSD/4 GBGB DDR4,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q,"Rs 185,000"
9,ACER ASPIRE 7 GAMING LAPTOP,WINDOWS 10 HOME,"15.6"" (1920 X 1080)",AMD Ryzen™ 5-5500U hexa-core | NA,512 GB SSD/8 GBGB DDR4,2.15,2.29 x 36.3 x 25.4,NVIDIA® GeForce® GTX 1650,"Rs 53,490"


In [102]:
#Saving dataset to csv
df.to_csv("Digit_Gaming_Laptops.csv")

#### Question 8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [131]:
driver=webdriver.Chrome(r'C:\Users\Ricky Riddhi\Downloads\chromedriver_win32\chromedriver.exe')

In [132]:
url = "https://www.forbes.com/"
driver.get(url)
time.sleep(3)

In [133]:
button = driver.find_element_by_xpath("//button[@class='icon--hamburger']")
button.click()
time.sleep(1)

In [134]:
#select billionaire  
bill = driver.find_element_by_xpath("/html/body/div[1]/header/nav/div[3]/ul/li[1]")
bill.click()
time.sleep(1)

In [135]:
#select world billionaire  
world_bill= driver.find_element_by_xpath("/html/body/div[1]/header/nav/div[3]/ul/li[1]/div[2]/ul/li[2]/a")
world_bill.click()
time.sleep(1)

In [136]:
#Scraping required data from the web page
#Creating empty lists
Rank = []
Person_Name = []
total_net_worth = []
Age = []
citizenship = []
Source = []
industry = []

In [137]:
while(True):
    #Scraping the data of rank of the billionaires
    rank_tags= driver.find_elements_by_xpath("//div[@class='rank']")
    for rank in rank_tags:
        Rank.append(rank.text)
    time.sleep(1)
    
    
    #Scraping the data of names of the billionaires
    name_tags= driver.find_elements_by_xpath("//div[@class='personName']/div")
    for name in name_tags:
        Person_Name.append(name.text)
    time.sleep(1)
    
    
    #Scraping data of age of the billionaires
    age_tags= driver.find_elements_by_xpath("//div[@class='age']/div")
    for age in age_tags:
        Age.append(age.text)   
    time.sleep(1)
    
    
    #Scraping data of citizenship of the billionaires
    cit_tags= driver.find_elements_by_xpath("//div[@class='countryOfCitizenship']")
    for cit in cit_tags:
        citizenship.append(cit.text)
    time.sleep(1)
    
    
    #Scraping data of source of income of the billionaires
    sour_tags= driver.find_elements_by_xpath("//div[@class='source']")
    for sour in sour_tags:
        Source.append(sour.text)    
    time.sleep(1)
    
    
    #Scraping data of Industry of the billionaires
    ind_tags= driver.find_elements_by_xpath("//div[@class='category']//div")
    for ind in ind_tags:
        industry.append(ind.text)
        
        
    #scraping data of net_worth of billionaires
    net_tags= driver.find_elements_by_xpath("//div[@class='netWorth']//div[1]")
    for net in net_tags:
        total_net_worth.append(net.text)
    time.sleep(1)
    
    
    #Clicking on next button
    try:
        next_button = driver.find_element_by_xpath("//button[@class='pagination-btn pagination-btn--next ']")
        next_button.click()
    except:
        break
        
#Scraping data of net worth
Net_Worth = []
for i in range(0,len(total_net_worth),2):
    Net_Worth.append(total_net_worth[i])

In [138]:
#Framing data
Billionaires=pd.DataFrame({})
Billionaires['Rank'] = Rank
Billionaires['Names'] = Person_Name
Billionaires['Net Worth'] = Net_Worth
Billionaires['Age'] = Age
Billionaires['Citizenship'] = citizenship
Billionaires['Source'] = Source
Billionaires['Industry'] = industry
Billionaires

,Rank,Names,Net Worth,Age,Citizenship,Source,Industry
0,1.,Jeff Bezos,$177 B,57,United States,Amazon,Technology
1,2.,Elon Musk,$151 B,49,United States,"Tesla, SpaceX",Automotive
2,3.,Bernard Arnault & family,$150 B,72,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$124 B,65,United States,Microsoft,Technology
4,5.,Mark Zuckerberg,$97 B,36,United States,Facebook,Technology
...,...,...,...,...,...,...,...
2750,2674.,Daniel Yong Zhang,$1 B,49,China,e-commerce,Technology
2751,2674.,Zhang Yuqiang,$1 B,65,China,Fiberglass,Manufacturing
2752,2674.,Zhao Meiguang,$1 B,58,China,gold mining,Metals & Mining
2753,2674.,Zhong Naixiong,$1 B,58,China,conglomerate,Diversified


In [139]:
#Saving dataset in csv
Billionaires.to_csv("Forbes_Billionaries.csv")

#### Question 9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [197]:
#Import all libraries
import selenium
from selenium import webdriver
import pandas as pd
import warnings
import requests

# importing regex
import re
from selenium import webdriver

# Importing required Exceptions
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait

In [198]:
driver=webdriver.Chrome(r'C:\Users\Ricky Riddhi\Downloads\chromedriver_win32\chromedriver.exe')

In [199]:
driver.get(url = "https://www.youtube.com/")
driver.maximize_window()
time.sleep(2)

In [200]:
close = driver.find_element_by_xpath('/html/body/ytd-app/div[1]/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[3]/ytd-banner-promo-renderer/div/ytd-button-renderer/a/yt-icon-button/button/yt-icon')

In [201]:
close.click()

In [202]:
#video url
url="https://www.youtube.com/watch"
driver.get(url)

In [207]:
i=0
while(i<100):
    driver.execute_script("window.scrollBy(0,5000)") # scroll down to get more comments
    i+=1
while(i<402):
    driver.execute_script("window.scrollBy(0,10000)") # scroll down to get more comments
    i+=1

In [210]:
#fetch comments tag

com_tags=driver.find_elements_by_xpath('//yt-formatted-string[@id="content-text"]')

comments=[]

for i in com_tags:
    comments.append(i.text)
len(comments)

19

In [211]:
upvote_tags=driver.find_elements_by_xpath('//span[@id="vote-count-middle"]')

upvotes=[]

for i in upvote_tags:
    try:
        upvotes.append(i.text)
    except:
        upvotes.append('---')
len(upvotes)

19

In [212]:
upvote_tags=driver.find_elements_by_xpath('//span[@id="vote-count-middle"]')

upvotes=[]

for i in upvote_tags:
    try:
        upvotes.append(i.text)
    except:
        upvotes.append('---')
len(upvotes)

19

In [213]:
ago_tags=driver.find_elements_by_xpath('//yt-formatted-string[@class="published-time-text above-comment style-scope ytd-comment-renderer"]//a')

ago=[]

for i in ago_tags:
    try:
        ago.append(i.text)
    except:
        ago.append('---')
len(ago)

19

In [214]:
# create dataframe

youtube_comments=pd.DataFrame()

youtube_comments['Comment']=comments
youtube_comments['Upvotes']=upvotes
youtube_comments['Time']=ago

youtube_comments.head(10)

,Comment,Upvotes,Time
0,Enough for refresh our mind 💐💜💐,16,1 month ago
1,💔Life mei kisi se pyar nahi Karna chahiye 💔,5,2 days ago
2,️❣️😘One' 0f the best thing about dude is that ...,5,1 month ago
3,जो भी इस वक्त मेरे इस comment पढ़ रहा है/रही ह...,133,2 days ago
4,"""""Yesterday is history ""\n""Tomorrow is mystery...",9,1 month ago
5,I Leave a Comment here so whenever someone lik...,4,1 month ago
6,"""Yesterday is history""\n ""tomorrow is mystery""...",8,1 month ago
7,Song: Beautiful\nVoice: Heart Touching Overall...,15,1 month ago
8,One of the best things about dude is that he n...,9,1 month ago
9,very nice.i love it...jubin's voice is relaxin...,9,1 month ago


#### Question 10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description

In [243]:
#Import all libraries
import selenium
from selenium import webdriver
import pandas as pd
import warnings
import requests

# importing regex
import re
from selenium import webdriver

# Importing required Exceptions
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait

In [244]:
driver=webdriver.Chrome(r'C:\Users\Ricky Riddhi\Downloads\chromedriver_win32\chromedriver.exe')

In [245]:
# Getting the web page of mentioned url
url = "https://www.hostelworld.com/"
driver.get(url)
time.sleep(5)

In [248]:
driver.find_element_by_id('location-text-input-field').click()
time.sleep(5)
driver.find_element_by_xpath('//*[@id="search-input-field"]').send_keys('London')
time.sleep(5)
#click on search
driver.find_element_by_xpath('//*[@id="predicted-search-results"]/li[2]').click()
driver.find_element_by_id('search-button').click()

In [249]:
hostel_name = []
distance = []
pvt_prices = []
dorms_price = []
rating = []
reviews = []
over_all = []
facilities = []
description =[]
product_url=[]

In [ ]:
# fetch the data

for i in driver.find_elements_by_xpath("//div[@class = 'pagination-item pagination-current']"):
    i.click()
    time.sleep(4)
    #fetching hostel name
    try:
        name = driver.find_elements_by_xpath("//h2[@class='title title-6']")
        for i in name:
            hostel_name.append(i.text)
    except NoSuchElementException:
        hostel_name.append('-')
    #fetching distance from city centre
    
    try:
        dist = driver.find_elements_by_xpath("//div[@class='subtitle body-3']//a//span[1]")
        for i in dist:
            distance.append(i.text.replace('Hostel - ',''))
    except NoSuchElementException:
        distance.append('-')
        
    for i in driver.find_elements_by_xpath("//div[@class='prices-col']"):
    #fetch privates from price
        try:
            pvt_price = driver.find_elements_by_xpath("//a[@class='prices']//div[1]//div")
            pvt_prices.append(pvt_price.text)
        except NoSuchElementException:
            pvt_prices.append('-')
    #fetching dorms from price
    for i in driver.find_elements_by_xpath("//div[@class='prices-col']"):
        try:
            dorms = driver.find_element_by_xpath("//a[@class='prices']//div[2]//div")
            dorms_price.append(dorms.text)
        except NoSuchElementException:
            dorms_price.append('-')
            #fetching facilities
    try:
        fac1 = driver.find_elements_by_xpath("//div[@class='has-wifi']")
        fac2 = driver.find_elements_by_xpath("//div[@class='has-sanitation']")
        for i in fac1:
            for j in fac2:
                facilities.append(i.text +', '+ j.text )
    except NoSuchElementException:
        facilities.append('-')
    #lets fetch url of each hostel
    p_url = driver.find_elements_by_xpath("//div[@class='prices-col']//a[2]")
    for i in p_url:
        product_url.append(i.get_attribute('href'))

for i in product_url:
    driver.get(i)
    time.sleep(3)
    #lets click on show more button for description
    try:
        more= driver.find_elements_by_xpath("//a[@class='toggle-content']")
        
        time.sleep(5)
    except NoSuchElementException:
        pass
    #fetching ratings
    try:
        rat = driver.find_element_by_xpath("//div[@class='score orange big' or @class='score gray big']")
        rating.append(rat.text)
    except NoSuchElementException:
        rating.append('-')
        
    #fetching total reviews
        
    try:
        rws = driver.find_element_by_xpath("//div[@class='reviews']")
        reviews.append(rws.text.replace('Total Reviews',''))
    except NoSuchElementException:
        reviews.append('-')
        #fetch overall review
    try:
        overall_rw = driver.find_element_by_xpath("//div[@class='keyword']//span")
        over_all.append(overall_rw.text)
    except NoSuchElementException:
        over_all.append('-')
    #fetch property description 
    try:
        disc = driver.find_element_by_xpath("//div[@class='content']")
        description.append(disc.text)
    except NoSuchElementException:
        over_all.append('-')

In [ ]:
len(hostel_name),len(distance),len(pvt_prices),len(dorms_price),len(rating),len(reviews),len(over_all),len(facilities),len(description)

In [221]:
hostel_name = hostel_name[0:81]
distance = distance[0:81]
pvt_prices = pvt_prices[0:81]
dorms_price = dorms_price[0:81]
rating = rating[0:81]
reviews = reviews[0:81]
over_all = over_all[0:81]
facilities = facilities[0:81]
description =description[0:81]
product_url = product_url[0:81]

In [222]:
Hostel = pd.DataFrame({})
Hostel['Hostel_Name'] = hostel_name
Hostel['Distance fron city centre'] = distance
Hostel['Ratings'] = rating
Hostel['Total_reviews'] = reviews
Hostel['Overall Reviews'] = over_all
Hostel['Privates from price'] = pvt_prices
Hostel['Dorms from price'] = dorms_price
Hostel['Description'] = description

In [223]:
Hostel

,Hostel_Name,Distance fron city centre,Ratings,Total_reviews,Overall Reviews,Privates from price,Dorms from price,Description
